In [1]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.applications import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from datetime import datetime
from glob import glob
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Set the image size
IMAGE_SIZE = [224, 224]

In [3]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Define dataset paths
train_path = '/content/drive/MyDrive/Underwater/dataset/Train'
validation_path = '/content/drive/MyDrive/Underwater/dataset/Validation'


In [5]:
# Load DenseNet121 with pre-trained weights
densenet = DenseNet121(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


29084464/29084464 [==============================] - 1s 0us/step


In [6]:
# Freeze layers
for layer in densenet.layers:
    layer.trainable = False


In [7]:
# Get the number of classes
folders = glob('/content/drive/MyDrive/Underwater/dataset/Train/*')
num_classes = len(folders)

In [8]:
# Create a custom neural network model using a pre-trained DenseNet base
x = Flatten()(densenet.output)
x = Dense(256, activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=densenet.input, outputs=prediction)


In [9]:
# Display a summary of the model architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [10]:
# Compile the model
adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [16]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [17]:
# Data augmentation for validation
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [18]:
# Create data generators
train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 2594 images belonging to 3 classes.


In [19]:
validation_set = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 583 images belonging to 3 classes.


In [20]:
#Set up model checkpoint
checkpoint = ModelCheckpoint(filepath='densenet_model_training.h5', monitor='accuracy', verbose=2, save_best_only=True, mode='max')
callbacks = [checkpoint]

# Training the model
start = datetime.now()
model_history = model.fit_generator(
    train_set,
    validation_data=validation_set,
    epochs=15,
    steps_per_epoch=5,
    validation_steps=32,
    callbacks=callbacks,
    verbose=2
)

duration = datetime.now() - start
print("Training completed in time: ", duration)

<ipython-input-20-bb5ab26db611>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history = model.fit_generator(


Epoch 1/15



Epoch 1: accuracy improved from -inf to 0.94375, saving model to densenet_model_training.h5
5/5 - 38s - loss: 0.6190 - accuracy: 0.9438 - val_loss: 0.9747 - val_accuracy: 0.9451 - 38s/epoch - 8s/step
Epoch 2/15

Epoch 2: accuracy improved from 0.94375 to 0.96250, saving model to densenet_model_training.h5
5/5 - 16s - loss: 0.3160 - accuracy: 0.9625 - 16s/epoch - 3s/step
Epoch 3/15

Epoch 3: accuracy did not improve from 0.96250
5/5 - 15s - loss: 0.8910 - accuracy: 0.9563 - 15s/epoch - 3s/step
Epoch 4/15

Epoch 4: accuracy improved from 0.96250 to 0.97500, saving model to densenet_model_training.h5
5/5 - 12s - loss: 0.4636 - accuracy: 0.9750 - 12s/epoch - 2s/step
Epoch 5/15

Epoch 5: accuracy did not improve from 0.97500
5/5 - 15s - loss: 1.4425 - accuracy: 0.9312 - 15s/epoch - 3s/step
Epoch 6/15

Epoch 6: accuracy did not improve from 0.97500
5/5 - 12s - loss: 1.3382 - accuracy: 0.9563 - 12s/epoch - 2s/step
Epoch 7/15

Epoch 7: accuracy improved from 0.97500 to 0.98750, saving model t